In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

output_dir = 'notebooks/XGBoost'

def standardize_region_names(df):
    """Стандартизация названий регионов"""
    df_clean = df.copy()
    
    replacements = {
        'Ненецкий авт.округ': 'Ненецкий автономный округ',
        'Hенецкий авт.округ': 'Ненецкий автономный округ',
        '  Ненецкий автономный округ': 'Ненецкий автономный округ',
        
        'Ямало-Ненецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        'Ямало-Hенецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        '  Ямало-Ненецкий автономный округ': 'Ямало-Ненецкий автономный округ',
        
        'Ханты-Мансийский авт.округ-Югра': 'Ханты-Мансийский автономный округ - Югра',
        '  Ханты-Мансийский автономный округ - Югра': 'Ханты-Мансийский автономный округ - Югра',
        
        'Республика Татарстан(Татарстан)': 'Республика Татарстан',
        'Чувашская Республика(Чувашия)': 'Чувашская Республика',
        'Республика Северная Осетия- Алания': 'Республика Северная Осетия-Алания',
        
        'Oмская область': 'Омская область',
        'Hижегородская область': 'Нижегородская область',
        
        'г. Севастополь': 'г.Севастополь',
        'г.Москва': 'г.Москва',
        'г.Санкт-Петербург': 'г.Санкт-Петербург',
        
        'Чукотский авт.округ': 'Чукотский автономный округ',
        'Чукотский автономный округ': 'Чукотский автономный округ'
    }
    
    df_clean['Регион'] = df_clean['Регион'].replace(replacements)
    df_clean['Регион'] = df_clean['Регион'].str.strip()
    
    return df_clean

class SKRXGBoostForecasterAdvanced:
    def __init__(self, random_state=42):
        self.model = None
        self.scaler = StandardScaler()
        self.feature_names = []
        self.is_fitted = False
        self.last_known_data = None
        self.random_state = random_state
        self.first_year = None
        self.validation_results = None
        
    def clean_numeric_columns(self, df):
        """
        Очистка числовых колонок
        """
        df = df.copy()
        
        numeric_columns = [col for col in df.columns if col not in ['Регион', 'Год', 'СКР']]
        
        for col in numeric_columns:
            if df[col].dtype == 'object':
                df[col] = (df[col]
                          .astype(str)
                          .str.replace('\xa0', '')
                          .str.replace(' ', '')
                          .str.replace(',', '.')
                          .str.replace('−', '-')
                          .str.replace('–', '-')
                          )
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        if 'СКР' in df.columns:
            df['СКР'] = pd.to_numeric(df['СКР'], errors='coerce')
        
        return df
        
    def prepare_features(self, df, is_training=True):
        """
        Подготовка признаков для СКР с сохранением данных с 2014 года
        """
        df = df.copy().sort_values(['Регион', 'Год'])
        
        if self.first_year is None:
            self.first_year = df['Год'].min()
        
        # Создание лаговых признаков
        if is_training or df['Год'].min() < 2024:
            df['lag1_СКР'] = df.groupby('Регион')['СКР'].shift(1)
            df['lag2_СКР'] = df.groupby('Регион')['СКР'].shift(2)
            df['lag1_Браков'] = df.groupby('Регион')['Браков'].shift(1)
            df['lag1_Разводов'] = df.groupby('Регион')['Разводов'].shift(1)
            df['lag1_Число родившихся'] = df.groupby('Регион')['Число родившихся'].shift(1)
            
            df['СКР_MA2'] = df.groupby('Регион')['СКР'].transform(lambda x: x.rolling(2, min_periods=1).mean())
            df['СКР_MA3'] = df.groupby('Регион')['СКР'].transform(lambda x: x.rolling(3, min_periods=1).mean())
        
        # Тренд времени
        df['year_trend'] = df['Год'] - self.first_year
        df['год_от_начала'] = df['Год'] - self.first_year
        
        # Демографические индикаторы на душу населения
        df['Браков_на_1000'] = df['Браков'] / df['Численность населения'] * 1000
        df['Разводов_на_1000'] = df['Разводов'] / df['Численность населения'] * 1000
        df['Родившихся_на_1000'] = df['Число родившихся'] / df['Численность населения'] * 1000
        df['Преступлений_на_1000'] = df['Кол-во преступлений'] / df['Численность населения'] * 1000
        
        # Жилищные условия
        if 'Введено в действие общей площади жилых домов на 1000 человек населения' in df.columns:
            df['жилье_на_1000'] = df['Введено в действие общей площади жилых домов на 1000 человек населения']
        
        # Индексы
        df['соотношение_браков_разводов'] = df['Браков'] / (df['Разводов'] + 1)
        df['Социально_экономический_индекс'] = (
            df['Средняя ЗП'] / df['Величина прожиточного минимума'] - 
            (df['Уровень бедности'] / 100)
        )
        
        # Стабильность семьи
        df['стабильность_семьи'] = df['соотношение_браков_разводов'] / (df['Уровень безработицы'] + 1)
        
        # Относительные изменения
        df['изменение_населения'] = df.groupby('Регион')['Численность населения'].pct_change()
        df['изменение_ВРП'] = df.groupby('Регион')['Валовой региональный продукт на душу населения (ОКВЭД 2)'].pct_change()
        df['изменение_браков'] = df.groupby('Регион')['Браков'].pct_change()
        df['изменение_рождаемости'] = df.groupby('Регион')['Число родившихся'].pct_change()
        
        if is_training:
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if col not in ['Год']:
                    df[col] = df.groupby('Регион')[col].transform(
                        lambda x: x.fillna(x.median()) if not x.isnull().all() else x
                    )
            
            df = df.fillna(df.median(numeric_only=True))
        
        return df
    
    def train_validate_split(self, df, train_end_year=2021, validation_year=2022, test_year=2023):
        """
        Разделение на обучающую, валидационную и тестовую выборки
        """
        train_mask = df['Год'] <= train_end_year
        val_mask = df['Год'] == validation_year
        test_mask = df['Год'] == test_year
        
        return train_mask, val_mask, test_mask
    
    def fit_with_validation(self, df, train_end_year=2021, validation_year=2022):
        """
        Обучение с валидацией на 2022 году
        """
        print(f"\n{'='*70}")
        print(f"ОБУЧЕНИЕ МОДЕЛИ СКР НА ДАННЫХ ДО {train_end_year} ГОДА")
        print(f"ВАЛИДАЦИЯ НА {validation_year} ГОДУ")
        print(f"{'='*70}")
        
        df_clean = self.clean_numeric_columns(df)
        df_processed = self.prepare_features(df_clean, is_training=True)
        
        train_mask, val_mask, _ = self.train_validate_split(df_processed, train_end_year, validation_year)
        
        # Определение признаков для СКР
        self.feature_names = [
            # Демографические
            'Численность населения', 
            'Число родившихся',
            'Браков', 
            'Разводов',
            
            # Социально-экономические
            'Введено в действие общей площади жилых домов на 1000 человек населения',
            'жилье_на_1000',
            'Кол-во преступлений', 
            'Уровень безработицы', 
            'Уровень бедности', 
            'Величина прожиточного минимума',
            'Валовой региональный продукт на душу населения (ОКВЭД 2)',
            'Средняя ЗП',
            
            # Лаговые признаки
            'lag1_СКР', 
            'lag2_СКР',
            'lag1_Браков', 
            'lag1_Разводов',
            'lag1_Число родившихся',
            
            # Скользящие средние
            'СКР_MA2',
            'СКР_MA3',
            
            # Тренд и время
            'year_trend',
            'год_от_начала',
            
            # Производные показатели
            'Браков_на_1000', 
            'Разводов_на_1000',
            'Родившихся_на_1000',
            'Преступлений_на_1000',
            'соотношение_браков_разводов',
            'Социально_экономический_индекс',
            'стабильность_семьи',
            'изменение_населения',
            'изменение_ВРП',
            'изменение_браков',
            'изменение_рождаемости'
        ]
        
        self.feature_names = [f for f in self.feature_names if f in df_processed.columns]
        print(f"Используется {len(self.feature_names)} признаков")
        
        # Подготовка данных
        X_train = df_processed[train_mask][self.feature_names]
        X_val = df_processed[val_mask][self.feature_names]
        y_train = df_processed[train_mask]['СКР']
        y_val = df_processed[val_mask]['СКР']
        
        print(f"Обучающая выборка: {X_train.shape} (годы: {df_processed[train_mask]['Год'].min()}-{train_end_year})")
        print(f"Валидационная выборка: {X_val.shape} (год: {validation_year})")
        
        # Масштабирование
        scale_features = [f for f in self.feature_names if not f.startswith(('lag', 'СКР_MA', 'изменение_', 'год_от_начала'))]
        X_train_scaled = X_train.copy()
        X_val_scaled = X_val.copy()
        
        X_train_scaled[scale_features] = self.scaler.fit_transform(X_train[scale_features])
        X_val_scaled[scale_features] = self.scaler.transform(X_val[scale_features])
        
        # Обучение модели с оптимизированными параметрами для СКР
        self.model = xgb.XGBRegressor(
            max_depth=7,
            learning_rate=0.04,
            n_estimators=350,
            random_state=self.random_state,
            n_jobs=-1,
            subsample=0.75,
            colsample_bytree=0.75,
            reg_alpha=0.05,
            reg_lambda=0.8
        )
        
        print("\nОбучение модели СКР...")
        self.model.fit(X_train_scaled, y_train)
        self.is_fitted = True
        
        # Сохраняем последние известные данные
        self.last_known_data = df_processed[df_processed['Год'] == train_end_year].copy()
        
        # Валидация
        y_val_pred = self.model.predict(X_val_scaled)
        
        self.validation_results = {
            'y_true': y_val,
            'y_pred': y_val_pred,
            'year': validation_year,
            'metrics': self._calculate_detailed_metrics(y_val, y_val_pred, "Валидация СКР")
        }
        
        print(f"\n{'='*70}")
        print("РЕЗУЛЬТАТЫ ВАЛИДАЦИИ СКР:")
        print(f"{'='*70}")
        self.print_metrics(self.validation_results['metrics'])
        
        # Кросс-валидация
        self.cross_validation(X_train_scaled, y_train)
        
        # Сохранение результатов
        self.results = {
            'X_train': X_train_scaled, 'X_val': X_val_scaled,
            'y_train': y_train, 'y_val': y_val, 'y_val_pred': y_val_pred,
            'df_processed': df_processed,
            'train_years': f"{df_processed[train_mask]['Год'].min()}-{train_end_year}",
            'validation_year': validation_year
        }
        
        return self
    
    def test_on_2023(self, df):
        """
        Тестирование модели СКР на 2023 году (реальные данные)
        """
        print(f"\n{'='*70}")
        print("ТЕСТИРОВАНИЕ МОДЕЛИ СКР НА 2023 ГОДУ")
        print(f"{'='*70}")
        
        df_clean = self.clean_numeric_columns(df)
        df_processed = self.prepare_features(df_clean, is_training=False)
        
        test_mask = df_processed['Год'] == 2023
        
        if test_mask.sum() == 0:
            print("Нет данных за 2023 год для тестирования!")
            return None
        
        X_test = df_processed[test_mask][self.feature_names]
        y_test = df_processed[test_mask]['СКР']
        
        print(f"Тестовая выборка: {X_test.shape} (год: 2023)")
        
        # Масштабирование
        scale_features = [f for f in self.feature_names if not f.startswith(('lag', 'СКР_MA', 'изменение_', 'год_от_начала'))]
        X_test_scaled = X_test.copy()
        X_test_scaled[scale_features] = self.scaler.transform(X_test[scale_features])
        
        # Прогноз
        y_test_pred = self.model.predict(X_test_scaled)
        
        # Расчет метрик
        test_metrics = self._calculate_detailed_metrics(y_test, y_test_pred, "Тестирование СКР на 2023")
        
        print(f"\n{'='*70}")
        print("РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ СКР НА 2023 ГОДУ:")
        print(f"{'='*70}")
        self.print_metrics(test_metrics)
        
        # Сравнение с валидацией
        if self.validation_results:
            print(f"\nСРАВНЕНИЕ С ВАЛИДАЦИЕЙ НА {self.validation_results['year']} ГОДУ:")
            print(f"RMSE валидация: {self.validation_results['metrics']['RMSE']:.4f}")
            print(f"RMSE тест 2023: {test_metrics['RMSE']:.4f}")
            diff_percent = (test_metrics['RMSE'] - self.validation_results['metrics']['RMSE']) / self.validation_results['metrics']['RMSE'] * 100
            print(f"Изменение ошибки: {diff_percent:+.2f}%")
        
        # Сохраняем результаты тестирования
        self.test_results = {
            'y_true': y_test,
            'y_pred': y_test_pred,
            'year': 2023,
            'metrics': test_metrics,
            'X_test': X_test_scaled,
            'df_test': df_processed[test_mask].copy()
        }
        
        # Обновляем последние известные данные до 2023 года
        self.last_known_data = df_processed[df_processed['Год'] == 2023].copy()
        
        return test_metrics
    
    def _calculate_detailed_metrics(self, y_true, y_pred, context=""):
        """Расчет детализированных метрик для СКР"""
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)
        
        # Дополнительные метрики
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        max_error = np.max(np.abs(y_true - y_pred))
        median_error = np.median(np.abs(y_true - y_pred))
        
        # Процентили ошибок
        error_percentiles = np.percentile(np.abs(y_true - y_pred), [25, 50, 75, 90, 95])
        
        # Статистика по СКР
        skr_stats = {
            'СКР_mean_true': y_true.mean(),
            'СКР_mean_pred': y_pred.mean(),
            'СКР_std_true': y_true.std(),
            'СКР_std_pred': y_pred.std(),
            'СКР_min_true': y_true.min(),
            'СКР_max_true': y_true.max(),
            'СКР_min_pred': y_pred.min(),
            'СКР_max_pred': y_pred.max()
        }
        
        return {
            'MSE': mse, 'MAE': mae, 'RMSE': rmse, 'R2': r2,
            'MAPE': mape, 'MaxError': max_error, 'MedianError': median_error,
            'Error_25p': error_percentiles[0], 'Error_50p': error_percentiles[1],
            'Error_75p': error_percentiles[2], 'Error_90p': error_percentiles[3],
            'Error_95p': error_percentiles[4],
            'СКР_stats': skr_stats,
            'Context': context
        }
    
    def print_metrics(self, metrics):
        """Красивый вывод метрик СКР"""
        print(f"RMSE: {metrics['RMSE']:.4f}")
        print(f"MAE: {metrics['MAE']:.4f}")
        print(f"R²: {metrics['R2']:.4f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        print(f"Средний СКР (реальный): {metrics['СКР_stats']['СКР_mean_true']:.3f}")
        print(f"Средний СКР (прогноз): {metrics['СКР_stats']['СКР_mean_pred']:.3f}")
        print(f"Разница средних: {metrics['СКР_stats']['СКР_mean_pred'] - metrics['СКР_stats']['СКР_mean_true']:+.3f}")
    
    def cross_validation(self, X_train, y_train, cv=5):
        """
        Кросс-валидация для СКР
        """
        print("\nКросс-валидация модели СКР:")
        
        temp_model = xgb.XGBRegressor(
            max_depth=7,
            learning_rate=0.04,
            n_estimators=200,
            random_state=self.random_state,
            n_jobs=-1,
            subsample=0.75,
            colsample_bytree=0.75
        )
        
        try:
            scores = cross_val_score(temp_model, X_train, y_train, 
                                   scoring='neg_mean_squared_error', cv=cv)
            rmse_scores = np.sqrt(-scores)
            print(f"Средний RMSE: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")
            print(f"Диапазон: {rmse_scores.min():.4f} - {rmse_scores.max():.4f}")
        except Exception as e:
            print(f"Ошибка при кросс-валидации: {e}")
    
    def predict_future_advanced(self, future_years=[2024, 2025]):
        """
        Продвинутый прогноз СКР на будущие периоды
        """
        if not self.is_fitted:
            print("Сначала обучите модель!")
            return None
        
        if self.last_known_data is None:
            print("Нет данных для прогноза!")
            return None
        
        print(f"\n{'='*70}")
        print(f"ПРОГНОЗ СКР НА {future_years} ГОДЫ")
        print(f"{'='*70}")
        
        all_predictions = []
        
        for year_idx, year in enumerate(future_years):
            print(f"\nПрогноз СКР на {year} год:")
            
            year_predictions = []
            current_data = self.last_known_data.copy()
            
            for _, last_row in current_data.iterrows():
                future_row = last_row.copy()
                future_row['Год'] = year
                future_row['year_trend'] = year - self.first_year
                future_row['год_от_начала'] = year - self.first_year
                
                # Коэффициенты роста/снижения для СКР (более консервативные)
                growth_factors = {
                    'Численность населения': 1.002,
                    'Число родившихся': 1.008,  # Небольшой рост рождаемости
                    'Браков': 1.012,  # Рост браков
                    'Разводов': 1.006,  # Умеренный рост разводов
                    'Валовой региональный продукт на душу населения (ОКВЭД 2)': 1.018,
                    'Средняя ЗП': 1.045,
                    'Величина прожиточного минимума': 1.035,
                    'Уровень бедности': 0.98,
                    'Уровень безработицы': 0.985,
                    'Кол-во преступлений': 0.99,
                    'Введено в действие общей площади жилых домов на 1000 человек населения': 1.02
                }
                
                for col, factor in growth_factors.items():
                    if col in future_row:
                        # Добавляем региональную вариацию
                        region_factor = 1.0 + np.random.normal(0, 0.008)
                        future_row[col] = last_row[col] * factor * region_factor
                
                # Лаги из предыдущего прогноза
                if year_idx == 0:
                    future_row['lag1_СКР'] = last_row['СКР']
                    future_row['lag2_СКР'] = last_row.get('lag1_СКР', last_row['СКР'])
                else:
                    # Для второго года прогноза используем прогноз первого года
                    prev_year_pred = all_predictions[-len(current_data)]['СКР_прогноз']
                    future_row['lag1_СКР'] = prev_year_pred
                    future_row['lag2_СКР'] = last_row['СКР']
                
                future_row['lag1_Браков'] = future_row['Браков']
                future_row['lag1_Разводов'] = future_row['Разводов']
                future_row['lag1_Число родившихся'] = future_row['Число родившихся']
                
                # Скользящие средние
                future_row['СКР_MA2'] = (future_row['lag1_СКР'] + future_row['lag2_СКР']) / 2
                future_row['СКР_MA3'] = (future_row['lag1_СКР'] + future_row['lag2_СКР'] + 
                                         last_row.get('lag2_СКР', future_row['lag2_СКР'])) / 3
                
                # Пересчет производных показателей
                future_row['Браков_на_1000'] = future_row['Браков'] / future_row['Численность населения'] * 1000
                future_row['Разводов_на_1000'] = future_row['Разводов'] / future_row['Численность населения'] * 1000
                future_row['Родившихся_на_1000'] = future_row['Число родившихся'] / future_row['Численность населения'] * 1000
                future_row['Преступлений_на_1000'] = future_row['Кол-во преступлений'] / future_row['Численность населения'] * 1000
                
                if 'жилье_на_1000' in future_row:
                    future_row['жилье_на_1000'] = future_row['Введено в действие общей площади жилых домов на 1000 человек населения']
                
                future_row['соотношение_браков_разводов'] = future_row['Браков'] / (future_row['Разводов'] + 1)
                future_row['Социально_экономический_индекс'] = (
                    future_row['Средняя ЗП'] / future_row['Величина прожиточного минимума'] - 
                    (future_row['Уровень бедности'] / 100)
                )
                future_row['стабильность_семьи'] = future_row['соотношение_браков_разводов'] / (future_row['Уровень безработицы'] + 1)
                future_row['изменение_населения'] = (future_row['Численность населения'] - last_row['Численность населения']) / last_row['Численность населения']
                future_row['изменение_ВРП'] = (future_row['Валовой региональный продукт на душу населения (ОКВЭД 2)'] - last_row['Валовой региональный продукт на душу населения (ОКВЭД 2)']) / last_row['Валовой региональный продукт на душу населения (ОКВЭД 2)']
                future_row['изменение_браков'] = (future_row['Браков'] - last_row['Браков']) / last_row['Браков']
                future_row['изменение_рождаемости'] = (future_row['Число родившихся'] - last_row['Число родившихся']) / last_row['Число родившихся']
                
                year_predictions.append(future_row)
            
            # Прогноз на конкретный год
            future_df = pd.DataFrame(year_predictions)
            X_future = future_df[self.feature_names]
            
            # Масштабирование
            scale_features = [f for f in self.feature_names if not f.startswith(('lag', 'СКР_MA', 'изменение_', 'год_от_начала'))]
            X_future_scaled = X_future.copy()
            X_future_scaled[scale_features] = self.scaler.transform(X_future[scale_features])
            
            # Прогноз СКР
            predictions = self.model.predict(X_future_scaled)
            
            # Сохраняем результаты в формате, понятном для сервера
            for i, (_, last_row) in enumerate(current_data.iterrows()):
                all_predictions.append({
                    'Регион': last_row['Регион'],
                    'Год': year,
                    'СКР_прогноз': predictions[i],  # Прогноз
                    'СКР_предыдущий': last_row['СКР'],  # Фактическое значение за 2023
                    'Изменение_СКР': predictions[i] - last_row['СКР'],
                    'СКР_базовый': last_row['СКР'],
                    'Процент_изменения': ((predictions[i] - last_row['СКР']) / last_row['СКР']) * 100
                })
            
            # Обновляем последние данные для следующего года прогноза
            future_df['СКР'] = predictions
            self.last_known_data = future_df.copy()
            
            # Статистика по году
            year_stats = all_predictions[-len(current_data):]
            avg_skr = np.mean([p['СКР_прогноз'] for p in year_stats])
            avg_change = np.mean([p['Изменение_СКР'] for p in year_stats])
            positive_changes = sum(1 for p in year_stats if p['Изменение_СКР'] > 0)
            
            print(f"  Средний СКР: {avg_skr:.3f}")
            print(f"  Среднее изменение: {avg_change:+.4f}")
            print(f"  Регионов с ростом СКР: {positive_changes}/{len(current_data)}")
            print(f"  Диапазон СКР: {min(p['СКР_прогноз'] for p in year_stats):.3f} - {max(p['СКР_прогноз'] for p in year_stats):.3f}")
        
        results_df = pd.DataFrame(all_predictions)
        
        # Анализ динамики
        self._analyze_skr_predictions_dynamics(results_df, future_years)
        
        return results_df
    
    def _analyze_skr_predictions_dynamics(self, predictions_df, years):
        """Анализ динамики прогнозов СКР"""
        print(f"\n{'='*70}")
        print("АНАЛИЗ ДИНАМИКИ ПРОГНОЗОВ СКР:")
        print(f"{'='*70}")
        
        # Первые 10 регионов для примера
        for region in predictions_df['Регион'].unique()[:10]:
            region_data = predictions_df[predictions_df['Регион'] == region].sort_values('Год')
            if len(region_data) > 1:
                print(f"\n{region}:")
                for _, row in region_data.iterrows():
                    print(f"  {row['Год']}: СКР = {row['СКР_прогноз']:.3f} "
                          f"(Δ: {row['Изменение_СКР']:+.3f}, "
                          f"{row['Процент_изменения']:+.1f}%)")
        
        # Общая статистика
        print(f"\nОБЩАЯ СТАТИСТИКА СКР ПО ГОДАМ:")
        for year in years:
            year_data = predictions_df[predictions_df['Год'] == year]
            if not year_data.empty:
                avg_skr = year_data['СКР_прогноз'].mean()
                max_skr = year_data['СКР_прогноз'].max()
                min_skr = year_data['СКР_прогноз'].min()
                std_skr = year_data['СКР_прогноз'].std()
                
                print(f"\n{year} год:")
                print(f"  Средний СКР: {avg_skr:.3f}")
                print(f"  Максимальный: {max_skr:.3f}")
                print(f"  Минимальный: {min_skr:.3f}")
                print(f"  Стандартное отклонение: {std_skr:.3f}")
                print(f"  Размах: {max_skr - min_skr:.3f}")
    
    def save_model(self, filepath=None):
        """Сохранение модели СКР"""
        if not self.is_fitted:
            print("Модель не обучена!")
            return False
        
        if filepath is None:
            filepath = os.path.join(output_dir, 'skr_xgboost_advanced_model.pkl')
        
        model_data = {
            'model': self.model,
            'scaler': self.scaler,
            'feature_names': self.feature_names,
            'first_year': self.first_year,
            'last_known_data': self.last_known_data,
            'validation_results': self.validation_results,
            'test_results': getattr(self, 'test_results', None),
            'random_state': self.random_state
        }
        
        joblib.dump(model_data, filepath)
        print(f"Модель СКР сохранена в {filepath}")
        return True
    
    def load_model(self, filepath=None):
        """Загрузка модели СКР"""
        if filepath is None:
            filepath = os.path.join(output_dir, 'skr_xgboost_advanced_model.pkl')
            
        if not os.path.exists(filepath):
            print(f"Файл модели {filepath} не найден!")
            return False
        
        model_data = joblib.load(filepath)
        self.model = model_data['model']
        self.scaler = model_data['scaler']
        self.feature_names = model_data['feature_names']
        self.first_year = model_data['first_year']
        self.last_known_data = model_data['last_known_data']
        self.validation_results = model_data['validation_results']
        self.test_results = model_data['test_results']
        self.random_state = model_data.get('random_state', 42)
        self.is_fitted = True
        
        print(f"Модель СКР загружена из {filepath}")
        return True
    
    def prepare_final_output_for_server(self, predictions_df, target_year=2024):
        """
        Подготовка финального файла в формате для сервера
        Только 4 поля: Регион, Год, СКР, predictions
        """
        # Фильтруем данные за нужный год
        year_data = predictions_df[predictions_df['Год'] == target_year]
        
        final_output = pd.DataFrame({
            'Регион': year_data['Регион'],
            'Год': year_data['Год'],
            'СКР': year_data['СКР_предыдущий'],  # Фактическое значение
            'predictions': year_data['СКР_прогноз']  # Прогноз
        })
        
        return final_output

# Главная функция для СКР
def run_skr_pipeline():
    """
    Запуск пайплайна для СКР с созданием predictions_afr.xlsx
    """
    print("="*80)
    print("ПАЙПЛАЙН ДЛЯ СКР (predictions_afr.xlsx)")
    print("="*80)
    
    # Создаем директорию для результатов
    os.makedirs(output_dir, exist_ok=True)
    
    # Загрузка данных
    print("Загрузка данных СКР...")
    try:
        df_skr = pd.read_excel('Финальный вариант/общая_СКР (2).xlsx')
    except FileNotFoundError:
        print("ОШИБКА: Файл 'общая_СКР (2).xlsx' не найден!")
        print("Убедитесь, что файл находится в папке 'Финальный вариант'")
        return None
    
    # Стандартизация регионов
    print("Стандартизация названий регионов...")
    df_skr = standardize_region_names(df_skr)
    print(f"Уникальных регионов: {df_skr['Регион'].nunique()}")
    print(f"Период данных: {df_skr['Год'].min()}-{df_skr['Год'].max()}")
    
    print(f"\n{'='*70}")
    print("АЛЬТЕРНАТИВНЫЙ ПОДХОД ДЛЯ БОЛЕЕ ТОЧНОГО ПРОГНОЗА СКР")
    print(f"{'='*70}")
    
    # Создание и обучение модели по альтернативному подходу
    skr_forecaster = SKRXGBoostForecasterAdvanced()
    
    # Шаг 1: Обучаем модель на данных до 2022 года, валидируем на 2022 году
    skr_forecaster.fit_with_validation(df_skr, train_end_year=2021, validation_year=2022)
    
    # Шаг 2: Тестируем модель на реальных данных 2023 года
    test_metrics = skr_forecaster.test_on_2023(df_skr)
    
    # Сохранение модели после тестирования
    skr_forecaster.save_model()
    
    # Шаг 3: Прогноз на 2024 и 2025 годы
    print(f"\n{'='*70}")
    print("СОЗДАНИЕ ПРОГНОЗА СКР НА 2024-2025 ГОДЫ")
    print(f"{'='*70}")
    
    future_predictions = skr_forecaster.predict_future_advanced([2024, 2025])
    
    if future_predictions is not None:
        # ГАРАНТИРОВАННОЕ СОЗДАНИЕ ФАЙЛА predictions_afr.xlsx для 2024 года
        print(f"\n{'='*70}")
        print("СОЗДАНИЕ ФАЙЛА ДЛЯ СЕРВЕРА: predictions_afr.xlsx")
        print(f"{'='*70}")
        
        # Подготовка данных для 2024 года в формате для сервера
        server_data_2024 = skr_forecaster.prepare_final_output_for_server(future_predictions, target_year=2024)
        
        # Путь к файлу для сервера
        predictions_filepath = os.path.join(output_dir, 'predictions_afr.xlsx')
        
        # Сохраняем файл
        server_data_2024.to_excel(predictions_filepath, index=False)
        print(f"Файл для сервера сохранен: {predictions_filepath}")
        print(f"Размер файла: {server_data_2024.shape[0]} строк, {server_data_2024.shape[1]} колонок")
        print(f"Год прогноза: {server_data_2024['Год'].iloc[0]}")
        print(f"Уникальных регионов: {server_data_2024['Регион'].nunique()}")
        
        # Проверка структуры файла
        print("\nСТРУКТУРА ФАЙЛА predictions_afr.xlsx:")
        print(f"Колонки: {list(server_data_2024.columns)}")
        print(f"Типы данных: {server_data_2024.dtypes.to_dict()}")
        
        # Пример данных
        print("\nПЕРВЫЕ 5 СТРОК ФАЙЛА:")
        print(server_data_2024.head())
        
        # Статистика
        print("\nСТАТИСТИКА ПРОГНОЗА СКР НА 2024:")
        print(f"Средний СКР (факт 2023): {server_data_2024['СКР'].mean():.3f}")
        print(f"Средний прогноз СКР (2024): {server_data_2024['predictions'].mean():.3f}")
        print(f"Средний рост: {server_data_2024['predictions'].mean() - server_data_2024['СКР'].mean():+.3f}")
        
        # Также создаем файл для 2025 года (дополнительно)
        server_data_2025 = skr_forecaster.prepare_final_output_for_server(future_predictions, target_year=2025)
        predictions_2025_filepath = os.path.join(output_dir, 'predictions_afr_2025.xlsx')
        server_data_2025.to_excel(predictions_2025_filepath, index=False)
        print(f"\n✓ Дополнительный файл для 2025 года: {predictions_2025_filepath}")
        
        # Сохраняем полные результаты для анализа
        full_results_filepath = os.path.join(output_dir, 'full_skr_predictions_2024_2025_advanced.csv')
        future_predictions.to_csv(full_results_filepath, index=False, encoding='utf-8-sig')
        print(f"✓ Полные результаты сохранены: {full_results_filepath}")
        
        print(f"\n{'='*70}")
        print("ПАЙПЛАЙН СКР УСПЕШНО ЗАВЕРШЕН!")
        print("Файл predictions_afr.xlsx готов для загрузки в Git")
        print(f"{'='*70}")
        
        return predictions_filepath
    
    return None

if __name__ == "__main__":
    # Запускаем пайплайн для СКР
    predictions_file = run_skr_pipeline()
    
    if predictions_file:
        print(f"\nПайплайн выполнен успешно!")
        print(f"Файл для сервера создан: {predictions_file}")
    else:
        print("\nПайплайн завершился с ошибкой")